In [4]:
import os
import json
from datetime import datetime

In [ ]:
os.path.abspath("results")

In [ ]:

# Path to the directory containing the worker data and the model names file
workers_data_dir = "/home/ubuntu/slam/human_eval/Method_B/all_human_data_method_B_peptalk"
model_names_file = "/home/ubuntu/slam/human_eval/Method_B/data/peptalk/distributed_combinations_pep_talk.json"
out_file = "/home/ubuntu/slam/human_eval/Method_B/output/pep_talk.json"

check_worker_set = [f"worker_{num}" for num in range(0,101)]
with open(model_names_file, "r") as file :
    dist_data=json.load(file)
data=None

In [22]:
out_file="../test.json"
def evaluate_responses(workers_data_dir, distribution_file, response_file):
    out_dataset = []
    criteria_set = ["clarity", "intelligence", "likability", "trustworthy", "overall", "feedback"]
    with open(distribution_file, "r") as file:
        distribution = json.load(file)
    with open(response_file, "r") as file:
        all_responses = json.load(file)
    # List all files in the given directory
    for filename in os.listdir(workers_data_dir):
        file_path = os.path.join(workers_data_dir, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                worker_data = json.load(file)
            if "question_index" in worker_data and worker_data["question_index"]==10:
                curr_distribution_set=distribution.get(worker_data["question_set_id"],None)
                if curr_distribution_set is not None:
                    curr_worker_data_set = []
                    # print(curr_distribution_set, worker_data)
                    for i, curr_worker_data in enumerate(worker_data["evals"]):
                        curr_set = curr_worker_data.get('question')
                        model_names=list(curr_set[1].keys())
                        model_a, response_a_id = model_names[0],curr_set[1][model_names[0]]
                        model_b, response_b_id = model_names[1],curr_set[1][model_names[1]]
                        response_a= all_responses.get(response_a_id,None)
                        response_b= all_responses.get(response_b_id,None)
                
                        curr_resp_contruct={"model_a":model_a,"response_a":response_a,"model_b":model_b,"response_b":response_b}
                        curr_resp_contruct.update(curr_worker_data["result"])
                        curr_resp_contruct.update({"prompt_id":curr_set[0]})
                        # print(json.dumps(curr_resp_contruct))
                        if i == 0:
                            start_time = datetime.fromtimestamp(worker_data["start_time"])
                        end_time = datetime.fromtimestamp(curr_worker_data["time"])
                        curr_resp_contruct.update({"time_taken":(end_time-start_time).total_seconds()})
                        start_time=end_time
                        curr_worker_data_set.append(curr_resp_contruct)
                    out_dataset.append({worker_data["worker_id"]:curr_worker_data_set})
                    # out_dataset.append(curr_distribution_set)
                    # out_dataset.append(worker_data)
                    # count += 1
    print(json.dumps(out_dataset))
    with open(out_file, "w") as f:
        json.dump(out_dataset, f, indent=4)
workers_data_dir = "../results"
distribution_file = "../.human_eval_config/distribution.json"
response_file = "../.human_eval_config/responses.json"
evaluate_responses(workers_data_dir, distribution_file,response_file=response_file)


[{"13836413": [{"model_a": "starling-lm_7b", "response_a": " Great job on completing key tasks like preparing for the team presentation, finishing the quarterly report, weight lifting, and grocery shopping last week! Today, focus on aligning with your goals of crushing the team presentation by stress testing the new mobile feature, planning the next feature prototype to stay ahead in work, proof reading the quarterly report, sending it out for feedback, syncing up with the team, scheduling car maintenance service, and attending the orthopedic appointment for your lower back. Your rituals of cardio, zero inbox, reviewing user activity logs, and daily medium scan help maintain good habits and responsibilities. Keep up the hard work, you're on track to finish this quarter strong and prioritize your physical health!", "model_b": "stablelm-zephyr_3b", "response_b": "Good morning! Here's a short briefing for today:\n\nYesterday, you completed the following tasks: prepare for the team present

In [ ]:
for i, eval in enumerate(data["evals"]):
  if i == 0:
    start_time = datetime.datetime.fromtimestamp(data["start_time"])
  else:
    start_time = datetime.datetime.fromtimestamp(data["evals"][i-1]["time"])
  end_time = datetime.datetime.fromtimestamp(eval["time"])
  time_taken = end_time - start_time
  print(f"Eval {i+1} time taken: {time_taken}")

In [3]:
import os
data_dir ="../data"
out_dir = "../results"

for filename in os.listdir(data_dir):
    filename.split(".")[0]

peptalk_responses
all_task_completed_responses
new_user_responses
recommendation_responses
first_task_responses


In [5]:
out_file="../test_prompt.json"
def format_responses_by_prompt(workers_data_dir, distribution_file, response_file):
    out_dataset = {}
    
    with open(distribution_file, "r") as file:
        distribution = json.load(file)
    with open(response_file, "r") as file:
        all_responses = json.load(file)
    for filename in os.listdir(workers_data_dir):
        file_path = os.path.join(workers_data_dir, filename)
        if os.path.isfile(file_path):
            try:
                with open(file_path, "r") as file:
                    worker_data = json.load(file)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
                continue
            if "question_index" in worker_data and worker_data["question_index"] == 10:
                curr_distribution_set = distribution.get(worker_data["question_set_id"], None)
                if curr_distribution_set is not None:
                    for i,curr_worker_data in enumerate(worker_data["evals"]):
                        curr_set = curr_worker_data.get("question")
                        prompt_id = curr_set[0]
                        model_names = list(curr_set[1].keys())
                        model_a, response_a_id = (model_names[0], curr_set[1][model_names[0]])
                        model_b, response_b_id = (model_names[1], curr_set[1][model_names[1]])
                        response_a = all_responses.get(response_a_id, None)
                        response_b = all_responses.get(response_b_id, None)

                        curr_resp_contruct = {
                            "model_a": model_a,
                            "response_a": response_a,
                            "model_b": model_b,
                            "response_b": response_b,
                            "worker_id": worker_data["worker_id"],
                        }
                        curr_resp_contruct.update(curr_worker_data["result"])
                        if i == 0:
                            start_time = datetime.fromtimestamp(
                                worker_data["start_time"]
                            )
                        end_time = datetime.fromtimestamp(curr_worker_data["time"])
                        curr_resp_contruct.update(
                            {"time_taken": (end_time - start_time).total_seconds()}
                        )
                        start_time = end_time
                        if prompt_id not in out_dataset:
                            out_dataset[prompt_id] = []
                        out_dataset[prompt_id].append(curr_resp_contruct)
    
    # Convert to list of dicts with prompt_id as a key for each entry, if needed
    formatted_output = [{"prompt_id": prompt_id, "responses": responses} for prompt_id, responses in out_dataset.items()]
    with open(out_file, "w") as f:
        json.dump(formatted_output, f, indent=4)
    return formatted_output
workers_data_dir = "../results"
distribution_file = "../.human_eval_config/distribution.json"
response_file = "../.human_eval_config/responses.json"
formatted_output = format_responses_by_prompt(workers_data_dir, distribution_file,response_file=response_file)


In [8]:
all_models=['zephyr_7b-beta', 'mistral_7b-instruct', 'stablelm-zephyr_3b', 'gpt-4', 'starling-lm_7b', 'orca2_7b', 'openchat_7b-v3.5', 'llama2_7b-chat', 'neural-chat_7b', 'vicuna_7b', 'orca-mini_3b']

In [13]:
formatted_output[0]

{'prompt_id': 'eb5f52c7-d80a-4415-b8bd-94afc211559a',
 'responses': [{'model_a': 'starling-lm_7b',
   'response_a': " Great job on completing key tasks like preparing for the team presentation, finishing the quarterly report, weight lifting, and grocery shopping last week! Today, focus on aligning with your goals of crushing the team presentation by stress testing the new mobile feature, planning the next feature prototype to stay ahead in work, proof reading the quarterly report, sending it out for feedback, syncing up with the team, scheduling car maintenance service, and attending the orthopedic appointment for your lower back. Your rituals of cardio, zero inbox, reviewing user activity logs, and daily medium scan help maintain good habits and responsibilities. Keep up the hard work, you're on track to finish this quarter strong and prioritize your physical health!",
   'model_b': 'stablelm-zephyr_3b',
   'response_b': "Good morning! Here's a short briefing for today:\n\nYesterday, 

In [11]:
criteria = [
        "clarity",
        "intelligence",
        "likability",
        "trustworthiness",
        "overall",
    ]

model_performance = {
    model: {
        criterion: {"wins": 0, "ties": 0, "losses": 0} for criterion in criteria
    }
    for model in all_models
}
preference_matrix = {
    criterion: {
        model: {
            other_model: 0 for other_model in all_models if other_model != model
        }
        for model in all_models
    }
    for criterion in criteria
}
for outputs in formatted_output[0]:
    print(outputs)
    for response in outputs['responses']:
            # print(response)
            model1 = response["model_a"]
            model2 = response["model_b"]
            for crit in criteria:
                result = response.get(crit)
                if result == "Response A":
                    model_performance[model1][crit]["wins"] += 1
                    model_performance[model2][crit]["losses"] += 1
                    preference_matrix[crit][model1][model2] += 1
                elif result == "Response B":
                    model_performance[model1][crit]["losses"] += 1
                    model_performance[model2][crit]["wins"] += 1
                    preference_matrix[crit][model2][model1] += 1
                else:
                    model_performance[model1][crit]["ties"] += 1
                    model_performance[model2][crit]["ties"] += 1
    print(model_performance)

prompt_id


TypeError: string indices must be integers, not 'str'

In [61]:
def get_unique_prompt_ids():
    """
    Returns a list of unique prompt ids
    """
    prompt_data_dir = os.path.abspath("../data")
    with open(os.path.abspath("../.human_eval_config/prompt_info.json"), "r") as file:
        prompt_info = json.load(file)
    prompt_info = [{prompt["prompt_id"]: prompt["prompt"]} for prompt in prompt_info]
    prompt_ids = {}
    for filename in os.listdir(prompt_data_dir):
        use_case = filename.split(".")[0]
        file_path = os.path.join(prompt_data_dir, filename)
        with open(file_path, "r") as file:
            prompt_data = json.load(file)["prompt"]
            for inv_prompt in prompt_info:
                if prompt_data == list(inv_prompt.values())[0]:
                    prompt_ids[list(inv_prompt.keys())[0]] = use_case
    return prompt_ids

In [62]:
get_unique_prompt_ids()

{'eb5f52c7-d80a-4415-b8bd-94afc211559a': 'peptalk_responses',
 '5e2b0442-a13f-4991-9406-95ae4fccaa0e': 'all_task_completed_responses',
 'dd16110b-32c4-497f-b78d-3ab1f8e79c2d': 'new_user_responses',
 '516cea79-e5e1-4ea8-a733-a3dea2eb50c7': 'recommendation_responses',
 'e3474732-8bf8-463d-8b57-b84fbeccf369': 'first_task_responses'}

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json
import time
# Load a pre-trained embedding model (e.g., sentence-transformers)
model = SentenceTransformer('all-MiniLM-L6-v2')

with open("../.human_eval_config/distribution.json","r") as fp:
    distribution = json.load(fp)
with open("../.human_eval_config/prompt_info.json","r") as fp:
    prompt_info = json.load(fp)
    # Create a mapping from prompt_id to the actual prompt content
prompt_id_to_content = {item['prompt_id']: item['prompt'] for item in prompt_info}
with open("../.human_eval_config/responses.json","r") as fp:
    responses_dict = json.load(fp)
with open("../.human_eval_config/models_responses.json","r") as fp:
    models_responses_dict = json.load(fp)

In [ ]:
# from nltk import ngrams
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import SmoothingFunction
# from torch import max,tensor
# from nltk.tokenize import word_tokenize
# from sentence_transformers import util
# def embed_sentence(sentence, model):
#     # Embeds a sentence using a sentence transformer model
#     # Modify as per your embedding model's requirements
#     return model.encode(sentence, convert_to_tensor=True)

# def compute_bleu_score(reference, candidate):
#     reference_tokens = word_tokenize(reference)
#     candidate_tokens = word_tokenize(candidate)
#     smoothie = SmoothingFunction().method4
#     bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)
#     return bleu_score 

# def semantic_bleu_score(references, candidates, model=SentenceTransformer('all-mpnet-base-v2'), ngram_size=4, scaling_factor=1, bleu_weight=0.5):
#     scores = []
    
#     for candidate in candidates:
#         # Generate n-grams for reference and candidate
#         ref_ngrams = list(ngrams(references.split(), n=ngram_size))
#         cand_ngrams = list(ngrams(candidate.split(), n=ngram_size))

#         # Check if n-grams are empty
#         if not ref_ngrams or not cand_ngrams:
#             print("Empty n-grams detected. Skipping this candidate.")
#             continue

#         # Embed n-grams and convert tensors to numpy arrays
#         ref_embeddings = [embed_sentence(' '.join(ngram), model).cpu().numpy() for ngram in ref_ngrams]
#         cand_embeddings = [embed_sentence(' '.join(ngram), model).cpu().numpy() for ngram in cand_ngrams]
        
#         # Convert to tensors
#         ref_tensor = tensor(ref_embeddings)
#         cand_tensor = tensor(cand_embeddings)
#         # Check tensor sizes
#         if ref_tensor.size(0) == 0 or cand_tensor.size(0) == 0:
#             print("Empty embeddings detected. Skipping this candidate.")
#             continue
#         cosine_scores = util.pytorch_cos_sim(cand_tensor, ref_tensor)
#         max_scores = max(cosine_scores, dim=1).values

#         sem_scores = max_scores.numpy() * scaling_factor

#         bleu_score = compute_bleu_score(references, candidate)
#         if isinstance(bleu_score, (list, np.ndarray)):
#             bleu_score = np.mean(bleu_score)
#         adjusted_bleu = np.mean(sem_scores) + bleu_weight * bleu_score
#         scores.append(adjusted_bleu)
#     return np.nanmean(scores) if scores else 0


In [16]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assumes responses_dict is a dictionary with response IDs as keys and actual response texts as values.

def calculate_combined_score(model_responses,anchor_responses_id, model):
    # Prepare the anchor responses
    anchor_responses_text = [responses_dict[resp_id] for resp_id in anchor_responses_id]
    anchor_embeddings = model.encode(anchor_responses_text, convert_to_tensor=True).cpu()

    # Initialize lists to store scores
    bleu_scores = []
    semantic_scores = []

    # Iterate through each model's responses
    for _, response_id in model_responses.items():
        model_response_text = responses_dict[response_id]
        
        # Calculate the BLEU score for each anchor response against the model response
        for anchor_response_text in anchor_responses_text:
            bleu_score = sentence_bleu([word_tokenize(anchor_response_text)],
                                       word_tokenize(model_response_text),
                                       smoothing_function=SmoothingFunction().method4)
            bleu_scores.append(bleu_score)

        # Calculate the semantic similarity score for the model response
        model_embedding = model.encode(model_response_text, convert_to_tensor=True).cpu()
        semantic_score = cosine_similarity(anchor_embeddings, model_embedding.reshape(1, -1)).flatten().mean()
        semantic_scores.append(semantic_score)

    # Combine BLEU and semantic scores
    combined_scores = [np.mean([b, s]) for b, s in zip(bleu_scores, semantic_scores)]
    best_score=0.0
    # Determine the best response based on the combined score
    best_score_idx = np.argmax(combined_scores)
    # print(f"best_score_idx: {best_score_idx}")
    # best_response_id = list(model_responses.keys())[best_score_idx]
    # best_model = model_responses[best_response_id]
    return best_score_idx, best_score

In [18]:

# Function to calculate embedding score
def calculate_embedding_score(responses, anchor_reponses_id, model):
    anchor_reponses_text = [responses_dict[resp_id] for resp_id in anchor_reponses_id]
    anchor_embeddings = model.encode(anchor_reponses_text, convert_to_tensor=True).cpu()
    response_texts = [responses_dict[resp_id] for resp_id in responses.values()]
    response_embeddings = model.encode(response_texts, convert_to_tensor=True).cpu()
    scores = []
    for anchor_embedding in anchor_embeddings:
        similarities = cosine_similarity(anchor_embedding.reshape(1, -1), response_embeddings).flatten()
        scores.append(similarities)
    average_scores = np.mean(scores, axis=0)
    
    best_response_idx = np.argmax(average_scores)
    best_model = list(responses.keys())[best_response_idx]
    best_score = average_scores[best_response_idx]
    # print(f"Best average score = {best_score}, best model = {best_model}")
    return best_response_idx, best_score

anchor_model = 'gpt-4'

# Iterate over the distribution
results = []
for id, (prompt_id, question_sets) in enumerate(distribution.items()):
    print(f"Count {id+1} of {len(distribution)} and id {list(distribution.keys())[id]}")
    worker_set= {
            "worker_id":id,
            "question_set_id":list(distribution.keys())[id],
            "evals":[],
            "start_time": time.time(),
            "end_time":0,
            "question_index": 0
        }
    
    evals = []
    for ind,question_set in enumerate(question_sets):
        prompt_id = question_set[0]
        responses = question_set[1]
        # print(question_set)
        prompt_content = prompt_id_to_content.get(prompt_id, "")
        # Calculate the best response based on embedding score
        anchor_reponses_id = models_responses_dict[prompt_id][anchor_model]
        best_model, best_score = calculate_combined_score(responses, anchor_reponses_id, model)
        # Append the result to the results list
        worker_set["evals"].append({
            "result": { 
                "overall": "Response A" if best_model==0 else "Response B",
                'feedback': "",
            },
            "time": time.time(),
            "question": question_set
        })
        worker_set["question_index"]+=1
    worker_set["end_time"] = time.time()
    with open(f"../sim_results_sem/{id}.json", "w") as output:
        json.dump(worker_set, output,indent=4)
    
    results.append(worker_set)


Count 1 of 250 and id 9f58d593-81c3-4346-b41e-e783d98ef744
best_score_idx: 0
best_score_idx: 0
best_score_idx: 1
best_score_idx: 0


best_score_idx: 1
best_score_idx: 0
best_score_idx: 1
best_score_idx: 0
best_score_idx: 0
best_score_idx: 1
Count 2 of 250 and id 3b86e138-b009-4cbf-8dae-cdfa3c8931d0
best_score_idx: 1
best_score_idx: 1
best_score_idx: 1
best_score_idx: 0
best_score_idx: 1
best_score_idx: 1
best_score_idx: 1
best_score_idx: 0
best_score_idx: 0
best_score_idx: 0
Count 3 of 250 and id 4109ae31-ca63-4a65-8cda-f1543702c9dc
best_score_idx: 0
best_score_idx: 0
best_score_idx: 0
best_score_idx: 1
best_score_idx: 0
best_score_idx: 1
best_score_idx: 0
best_score_idx: 0
best_score_idx: 1
best_score_idx: 0
Count 4 of 250 and id d752eb34-9de9-4951-85f1-02740874113c
best_score_idx: 0
best_score_idx: 1
best_score_idx: 0
best_score_idx: 1
best_score_idx: 1
best_score_idx: 0
best_score_idx: 0
best_score_idx: 0
best_score_idx: 1
best_score_idx: 1
Count 5 of 250 and id 179c8d8b-df95-4709-9e51-1811699282d2
best_score_idx: 0
best_score_idx: 1
best_score_idx: 1
best_score_idx: 1
best_score_idx: 1
best_score_idx: 0
best_sco

In [13]:
with open("results_sim.json", "w") as results_file:
    json.dump(results, results_file, indent=4)

In [21]:
results_df

NameError: name 'results_df' is not defined